# 라이브러리 호출

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 데이터 불러오기

In [204]:
train = pd.read_csv("train.csv")

In [90]:
test = pd.read_csv("test.csv")

In [91]:
sub = pd.read_csv("sample_submission.csv")

### 1.컬럼 명 파악
- PassengerId - 승객의 고유 ID : gggg - pp = 그룹과 고유인원의 ID 
- HomePlanet - 거주하고 있는 행성(또는 출발한 행성)
- CryoSleep - 냉동수면 여부
- Cabin - 객실 정보(구역/번호/방향)
- Destination - 목적지
- Age - 나이 
- VIP - VIP 티켓 여부
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - 선박 내에서 편의시설을 이용한 금액
- name - 승객 이름

In [ ]:
obj_col = ["PassengerId", "HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP"]

In [ ]:
for col in obj_col :
    print(train[col].unique())

['0001_01' '0002_01' '0003_01' ... '9279_01' '9280_01' '9280_02']
['Europa' 'Earth' 'Mars' nan]
[False True nan]
['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']
['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]
[False True nan]


In [ ]:
train.groupby(["VIP","Transported"]).size()

VIP    Transported
False  False          4093
       True           4198
True   False           123
       True             76
dtype: int64

# EDA

#### VIP별, 출발행성 별 분석하기

In [ ]:
train.groupby(["VIP","HomePlanet"]).size()

VIP    HomePlanet
False  Earth         4487
       Europa        1958
       Mars          1653
True   Europa         131
       Mars            63
dtype: int64

#### PassengerId에서 그룹을 나누어 출발지역이 같은지 확인

In [161]:
train["Group"] = train["PassengerId"].str[:4]

In [162]:
home_group = train.groupby(["Group","HomePlanet"]).size().reset_index()

In [ ]:
home_group[home_group["Group"].duplicated()]

,Group,HomePlanet,0


In [ ]:
train[train["HomePlanet"].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
59,0064_02,NaN,True,E/3/S,TRAPPIST-1e,33.0,False,0.0,0.0,NaN,0.0,0.0,Colatz Keen,True
113,0119_01,NaN,False,A/0/P,TRAPPIST-1e,39.0,False,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,False
186,0210_01,NaN,True,D/6/P,55 Cancri e,24.0,False,0.0,0.0,NaN,0.0,0.0,Arraid Inicont,True
225,0242_01,NaN,False,F/46/S,TRAPPIST-1e,18.0,False,313.0,1.0,691.0,283.0,0.0,Almone Sté,False
234,0251_01,NaN,True,C/11/S,55 Cancri e,54.0,False,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9084_01,NaN,False,E/582/P,TRAPPIST-1e,25.0,False,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,False
8613,9194_01,NaN,False,E/603/S,55 Cancri e,53.0,False,0.0,4017.0,0.0,13.0,3147.0,NaN,False
8666,9248_01,NaN,False,F/1792/S,55 Cancri e,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,True
8674,9257_01,NaN,False,F/1892/P,TRAPPIST-1e,13.0,False,39.0,0.0,1085.0,24.0,0.0,Ties Apple,False


In [163]:
mapping = {}
for i,j in zip(home_group["Group"], home_group["HomePlanet"]) : 
    mapping[i] = j 

In [ ]:
train["HomePlanet"].isnull().sum()

111

In [ ]:
train[train["HomePlanet"].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group
186,0210_01,NaN,True,D/6/P,55 Cancri e,24.0,False,0.0,0.0,NaN,0.0,0.0,Arraid Inicont,True,0210
225,0242_01,NaN,False,F/46/S,TRAPPIST-1e,18.0,False,313.0,1.0,691.0,283.0,0.0,Almone Sté,False,0242
234,0251_01,NaN,True,C/11/S,55 Cancri e,54.0,False,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,True,0251
274,0303_01,NaN,True,G/41/S,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,Oraryn Kirklander,True,0303
286,0315_01,NaN,True,G/42/S,PSO J318.5-22,35.0,False,0.0,0.0,0.0,0.0,0.0,Adriet Valezaley,True,0315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8468,9043_01,NaN,True,F/1848/P,TRAPPIST-1e,25.0,False,0.0,0.0,0.0,0.0,0.0,Cobix Erle,True,9043
8515,9084_01,NaN,False,E/582/P,TRAPPIST-1e,25.0,False,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,False,9084
8666,9248_01,NaN,False,F/1792/S,55 Cancri e,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,True,9248
8674,9257_01,NaN,False,F/1892/P,TRAPPIST-1e,13.0,False,39.0,0.0,1085.0,24.0,0.0,Ties Apple,False,9257


#### Group이 n 일 때 HomePlanet의 null 값을 채우기

In [164]:
train["HomePlanet"].fillna(train["Group"].map(mapping), inplace = True)

In [165]:
train["last_name"] = train["Name"].str.split(" ").str[-1]
# train["Name"].str.split(" ").str.get(1)

In [166]:
name_group = train.groupby(["last_name", "HomePlanet"]).size().reset_index()

In [167]:
mapping2 = {}
for i,j in zip(name_group["last_name"], name_group["HomePlanet"]) : 
    mapping2[i] = j 

In [168]:
train["HomePlanet"].fillna(train["last_name"].map(mapping2), inplace = True)

In [177]:
mapping3 = {"55 Cancri e" : "Europa",
           "PSO J318.5-22" : "Earth	",
           "TRAPPIST-1e" : "Earth"}

In [178]:
train["HomePlanet"].fillna(train["Destination"].map(mapping3), inplace = True)

#### VIP 확인해보기

In [183]:
train["VIP"].value_counts()

False    8291
False     203
True      199
Name: VIP, dtype: int64

In [184]:
train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP               0
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
Group             0
last_name       200
dtype: int64

## paybill


In [185]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported', 'Group', 'last_name'],
      dtype='object')

In [192]:
train["fee"] = train["RoomService"] + train["FoodCourt"] + train["ShoppingMall"] + train["Spa"] + train["VRDeck"]

In [187]:
train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP               0
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
Group             0
last_name       200
fee             908
dtype: int64

In [188]:
fee_list = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

In [191]:
for col in fee_list : 
    train[col].fillna(train[col].median(), inplace = True)

In [193]:
train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
Group             0
last_name       200
fee               0
dtype: int64

## cabin

In [200]:
train["deck"] = train["Cabin"].str.split("/").str[0]

In [201]:
train["side"] = train["Cabin"].str.split("/").str[-1]

In [229]:
train.groupby(["HomePlanet", "deck"]).size()

HomePlanet  deck
Earth       D          1
            E        404
            F       1652
            G       2559
Europa      A        256
            B        779
            C        747
            D        192
            E        133
            T          5
Mars        D        285
            E        339
            F       1142
dtype: int64

In [232]:
train["deck"].fillna(train["HomePlanet"].map({"Earth" : "G", "Europa" : "B", "Mars" : "F"}), inplace = True)

In [233]:
train.isnull().sum()

PassengerId      0
HomePlanet       0
CryoSleep      217
Cabin          199
Destination    182
Age            179
VIP              0
Name           200
Transported      0
Group            0
last_name      200
fee              0
deck             0
side           199
dtype: int64

In [238]:
side_group = train.groupby(["Group","side"]).size().reset_index()

In [244]:
mapping4 = {}
for i,j in zip(side_group["Group"], side_group["side"]) :
    mapping4[i] = j

In [252]:
train[train["side"].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,Name,Transported,Group,last_name,fee,deck,side
15,0012_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,Justie Pooles,False,0012,Pooles,908.0,G,NaN
93,0101_01,Mars,True,NaN,TRAPPIST-1e,31.0,False,Book Trad,True,0101,Trad,0.0,F,NaN
222,0239_01,Mars,False,NaN,TRAPPIST-1e,37.0,False,Diedow Resty,False,0239,Resty,1048.0,F,NaN
272,0301_01,Earth,True,NaN,TRAPPIST-1e,16.0,False,Margia Wriggins,False,0301,Wriggins,0.0,G,NaN
280,0310_01,Europa,False,NaN,TRAPPIST-1e,67.0,False,Naviton Coudered,False,0310,Coudered,4947.0,B,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,8375_01,Earth,False,NaN,PSO J318.5-22,15.0,False,Carlen Valezaley,True,8375,Valezaley,704.0,G,NaN
7899,8437_01,Earth,False,NaN,PSO J318.5-22,36.0,False,Anny Tersony,False,8437,Tersony,694.0,G,NaN
8039,8603_01,Earth,True,NaN,PSO J318.5-22,18.0,False,Hilip Klinson,False,8603,Klinson,0.0,G,NaN
8066,8623_01,Earth,False,NaN,TRAPPIST-1e,15.0,False,Vandy Hoffergess,False,8623,Hoffergess,1362.0,G,NaN


In [250]:
train["side"].fillna(train["Group"].map(mapping4), inplace = True)

In [251]:
train.isnull().sum()

PassengerId      0
HomePlanet       0
CryoSleep      217
Cabin          199
Destination    182
Age            179
VIP              0
Name           200
Transported      0
Group            0
last_name      200
fee              0
deck             0
side            99
dtype: int64

In [253]:
train[train["side"].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,Name,Transported,Group,last_name,fee,deck,side
15,0012_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,Justie Pooles,False,0012,Pooles,908.0,G,NaN
93,0101_01,Mars,True,NaN,TRAPPIST-1e,31.0,False,Book Trad,True,0101,Trad,0.0,F,NaN
222,0239_01,Mars,False,NaN,TRAPPIST-1e,37.0,False,Diedow Resty,False,0239,Resty,1048.0,F,NaN
272,0301_01,Earth,True,NaN,TRAPPIST-1e,16.0,False,Margia Wriggins,False,0301,Wriggins,0.0,G,NaN
280,0310_01,Europa,False,NaN,TRAPPIST-1e,67.0,False,Naviton Coudered,False,0310,Coudered,4947.0,B,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,8375_01,Earth,False,NaN,PSO J318.5-22,15.0,False,Carlen Valezaley,True,8375,Valezaley,704.0,G,NaN
7899,8437_01,Earth,False,NaN,PSO J318.5-22,36.0,False,Anny Tersony,False,8437,Tersony,694.0,G,NaN
8039,8603_01,Earth,True,NaN,PSO J318.5-22,18.0,False,Hilip Klinson,False,8603,Klinson,0.0,G,NaN
8066,8623_01,Earth,False,NaN,TRAPPIST-1e,15.0,False,Vandy Hoffergess,False,8623,Hoffergess,1362.0,G,NaN


## Destination

In [261]:
dest_group = train.groupby(["Group", "Destination"]).size().reset_index()

In [263]:
dest_group = dest_group.sort_values(0, ascending = False)

In [270]:
dest_group = dest_group.drop_duplicates()

In [280]:
mapping5 = {}
for i, j in zip(dest_group["Group"], dest_group["Destination"]) :
    mapping5[i] = j 

In [281]:
train["Destination"].fillna(train["Group"].map(mapping5), inplace = True)

In [282]:
train["Destination"].value_counts()

TRAPPIST-1e      5964
55 Cancri e      1820
PSO J318.5-22     806
Name: Destination, dtype: int64

In [284]:
train["Destination"].fillna("TRAPPIST-1e", inplace = True)

In [285]:
train.isnull().sum()

PassengerId      0
HomePlanet       0
CryoSleep      217
Cabin          199
Destination      0
Age            179
VIP              0
Name           200
Transported      0
Group            0
last_name      200
fee              0
deck             0
side             0
dtype: int64

## CryoSleep

In [290]:
train.isnull().sum()

PassengerId      0
HomePlanet       0
CryoSleep      217
Cabin          199
Destination      0
Age            179
VIP              0
Name           200
Transported      0
Group            0
last_name      200
fee              0
deck             0
side             0
dtype: int64

In [291]:
train[train["CryoSleep"].isnull()]["fee"]

92         0.0
98       703.0
104     2018.0
111        0.0
152      990.0
         ...  
8620       0.0
8651       0.0
8664       0.0
8675    2056.0
8687    3540.0
Name: fee, Length: 217, dtype: float64

In [293]:
train["CryoSleep"] = np.where(train["fee"] > 0, False, True)

In [295]:
train.isnull().sum()

PassengerId      0
HomePlanet       0
CryoSleep        0
Cabin          199
Destination      0
Age            179
VIP              0
Name           200
Transported      0
Group            0
last_name      200
fee              0
deck             0
side             0
dtype: int64

## Age

In [296]:
train["Age"].fillna(train["Age"].median(), inplace = True)

In [297]:
train.isnull().sum()

PassengerId      0
HomePlanet       0
CryoSleep        0
Cabin          199
Destination      0
Age              0
VIP              0
Name           200
Transported      0
Group            0
last_name      200
fee              0
deck             0
side             0
dtype: int64

# 전처리

### HomePlanet

In [211]:
train["Group"] = train["PassengerId"].str[:4]

In [356]:
mapping = {}
for i,j in zip(home_group["Group"], home_group["HomePlanet"]) : 
    mapping[i] = j 

In [213]:
home_group = train.groupby(["Group","HomePlanet"]).size().reset_index()

In [214]:
train["HomePlanet"].fillna(train["Group"].map(mapping), inplace = True)

In [215]:
train["last_name"] = train["Name"].str.split(" ").str[-1]
# train["Name"].str.split(" ").str.get(1)

In [216]:
name_group = train.groupby(["last_name", "HomePlanet"]).size().reset_index()

In [217]:
mapping2 = {}
for i,j in zip(name_group["last_name"], name_group["HomePlanet"]) : 
    mapping2[i] = j 

In [218]:
train["HomePlanet"].fillna(train["last_name"].map(mapping2), inplace = True)

In [219]:
mapping3 = {"55 Cancri e" : "Europa",
           "PSO J318.5-22" : "Earth",
           "TRAPPIST-1e" : "Earth"}

In [220]:
train["HomePlanet"].fillna(train["Destination"].map(mapping3), inplace = True)

### VIP

In [221]:
train["VIP"].fillna("False", inplace = True)

### bill

In [222]:
train["fee"] = train["RoomService"] + train["FoodCourt"] + train["ShoppingMall"] + train["Spa"] + train["VRDeck"]

In [223]:
fee_list = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

In [224]:
for col in fee_list : 
    train[col].fillna(train[col].median(), inplace = True)

In [225]:
train["fee"] = train["RoomService"] + train["FoodCourt"] + train["ShoppingMall"] + train["Spa"] + train["VRDeck"]

In [226]:
train.drop(["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"], axis = 1, inplace = True)

### Cabin

In [227]:
train["deck"] = train["Cabin"].str.split("/").str[0]

In [228]:
train["side"] = train["Cabin"].str.split("/").str[-1]

In [232]:
train["deck"].fillna(train["HomePlanet"].map({"Earth" : "G", "Europa" : "B", "Mars" : "F"}), inplace = True)

In [238]:
side_group = train.groupby(["Group","side"]).size().reset_index()

In [244]:
mapping4 = {}
for i,j in zip(side_group["Group"], side_group["side"]) :
    mapping4[i] = j

In [250]:
train["side"].fillna(train["Group"].map(mapping4), inplace = True)

In [256]:
train["side"].fillna("S", inplace = True)

## Destination

In [261]:
dest_group = train.groupby(["Group", "Destination"]).size().reset_index()

In [263]:
dest_group = dest_group.sort_values(0, ascending = False)

In [270]:
dest_group = dest_group.drop_duplicates()

In [280]:
mapping5 = {}
for i, j in zip(dest_group["Group"], dest_group["Destination"]) :
    mapping5[i] = j 

In [281]:
train["Destination"].fillna(train["Group"].map(mapping5), inplace = True)

In [284]:
train["Destination"].fillna("TRAPPIST-1e", inplace = True)

## CryoSleep

In [293]:
train["CryoSleep"] = np.where(train["fee"] > 0, False, True)

## AGE

In [296]:
train["Age"].fillna(train["Age"].median(), inplace = True)

# feature에 사용되지 않는 컬럼 삭제

In [298]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'Name', 'Transported', 'Group', 'last_name', 'fee', 'deck',
       'side'],
      dtype='object')

In [299]:
del_col = ["PassengerId", "Name", "last_name", "Cabin"]

In [301]:
train = train.drop(del_col, axis = 1)

# 원 핫 인코딩

In [304]:
from sklearn.preprocessing import LabelEncoder

In [330]:
train["VIP"] = train["VIP"].astype(bool)

In [325]:
obj_col = []
for i,j in zip(train.columns, train.dtypes) :
    if j == "object" :
        obj_col.append(i)

In [322]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HomePlanet   8693 non-null   int32  
 1   CryoSleep    8693 non-null   bool   
 2   Destination  8693 non-null   int32  
 3   Age          8693 non-null   float64
 4   VIP          8693 non-null   object 
 5   Transported  8693 non-null   bool   
 6   Group        8693 non-null   object 
 7   fee          8693 non-null   float64
 8   deck         8693 non-null   object 
 9   side         8693 non-null   object 
dtypes: bool(2), float64(2), int32(2), object(4)
memory usage: 492.5+ KB


In [333]:
for col in obj_col :
    label = LabelEncoder()
    train[col] = label.fit_transform(train[col])

# 모델 학습

In [350]:
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

In [346]:
x = train.drop("Transported", axis = 1)

In [347]:
y = train["Transported"]

In [348]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state=42)

In [351]:
model1 = XGBClassifier()
model2 = lgb.LGBMClassifier()
model3 = RandomForestClassifier()
model_list = [model1,model2,model3]

In [359]:
for model in model_list :
    model.fit(x_train, y_train)
    print("#" * 100)
    print(f"트레인 {model} : {model.score(x_train, y_train)}")
    print(f" 테스트 {model} : {model.score(x_test, y_test)}")

####################################################################################################
트레인 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...) : 0.9060972102387115
 테스트 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              ea

In [360]:
import pickle

In [365]:
with open("model.pkl", "wb") as f :
    pickle.dump(model2, f)

In [ ]:
for col in obj_col :
    label = LabelEncoder()
    loan[col] = label.fit_transform(loan[col])
    filename = f"label_{col}.pkl"
    with open(f'./pkl/{filename}', "wb") as f : 
        pickle.dump(label, f)